# Establish All Files

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import sys
sys.path.append('/content/gdrive/MyDrive/SKRIPSI')

# Installation Library

In [3]:
# I specified the nb_path to install the library in it
!pip install -U sentence-transformers
!pip install transformers
!pip install sentencepiece
!pip install redis
!pip install pytorch-lightning
!pip install lightning
!pip install pyngrok
!pip install rouge-score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 6.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

# Initiate Library

In [4]:
from ftt5_v3 import ParaphraseGenerator as ParaphraseGeneratorSmall
from ftt5_v4 import ParaphraseGenerator as ParaphraseGeneratorBase
import json
import time
import re
import numpy as np
import pandas as pd
import redis
import requests
import random

from flask import Flask, request, jsonify, render_template

from transformers import T5Tokenizer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer

from redis.commands.search.field import (
    NumericField,
    TagField,
    TextField,
    VectorField,
)
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.query import Query
from sentence_transformers import SentenceTransformer

# Main Program Library
from pyngrok import ngrok
from flask import Flask, flash, request, redirect, url_for, render_template
import joblib

In [5]:
def listToTxt(items, fileName):
  file = open(fileName,'w')
  for item in items:
    file.write(item +"\n")
  file.close()

def txtToList(source_file_path):
  new_yolo = []
  with open (source_file_path) as new_txt:
    yolo = new_txt.readlines()
  for x in yolo: new_yolo.append(x.strip())
  return new_yolo

entity = txtToList("/content/gdrive/MyDrive/SKRIPSI/pre_processing_data/OW-new_name_v2.txt")
entity = [s.upper() for s in entity]
entity = list(set(entity))

exception = [".", ",", "?", "!", "#", ")", "("]
print(entity[:10])

def add_brackets(text):
    # Remove leading whitespace
    text = text.lstrip()

    # Create a regular expression pattern to match any word in the lexicon (case-insensitive)
    pattern = re.compile(r'\b(' + '|'.join(re.escape(word) for word in entity) + r')\b', re.IGNORECASE)

    # Replace each match with the word surrounded by square brackets
    result = pattern.sub(lambda m: f'[{m.group(0).upper()}]', text)

    # Add a period at the end if the last character is not a punctuation mark
    if not re.search(r'[.!?]$', result):
        result += '.'

    return result

def remove_brackets(text):
    # Create a regular expression pattern to match any word surrounded by square brackets
    pattern = re.compile(r'\[([^\]]+)\]')

    # Replace each match with the word inside the brackets
    result = pattern.sub(lambda m: m.group(1), text)

    return result

text_with_brackets = "Jika [PEMKAB] mau membayarkan [UKT] mahasiswa satu per penerima beasiswa seperti prosedur membayar [UKT] di sister."
print(remove_brackets(text_with_brackets))

add_brackets("Jika pemkab mau membayarkan UKT mahasiswa satu per penerima beasiswa seperti prosedur membayar ukt di sister")

['QILBA', 'TIYANESYA', 'NOVIYANTI', 'HUMANITER', 'MAGVIRA', 'JSTOR', 'ERDA', 'VIENTY', 'CBET', 'ARIYANTO']
Jika PEMKAB mau membayarkan UKT mahasiswa satu per penerima beasiswa seperti prosedur membayar UKT di sister.


'Jika pemkab mau membayarkan [UKT] mahasiswa satu per penerima beasiswa seperti prosedur membayar [UKT] di [SISTER].'

In [6]:
tokenizer = T5Tokenizer.from_pretrained('t5-small')

def search_similarity(paraphrased_question, original_question):
  #define array
  original_question = original_question
  paraphrased_question = paraphrased_question

  # calculate dot product
  dot_product = np.dot(paraphrased_question, original_question)

  # calculate magnitude
  magnitude_a = np.linalg.norm(paraphrased_question)
  magnitude_b = np.linalg.norm(original_question)

  # calculate similarity between two sentence
  cosine_similarity = dot_product / (magnitude_a * magnitude_b)
  return cosine_similarity

def bleu(sentence1, sentence2):
  reference_tokens = [sentence1.split()]
  candidate_tokens = sentence2.split()
  smoothie = SmoothingFunction().method1
  bleu_score1 = sentence_bleu(reference_tokens, candidate_tokens, weights=(0.5, 0.5), smoothing_function=smoothie)
  bleu_score2 = sentence_bleu(reference_tokens, candidate_tokens, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smoothie)
  return bleu_score1, bleu_score2

def rouge(sentence1, sentence2):
  scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
  rouge_score = scorer.score(sentence1, sentence2)
  rouge_score1 = rouge_score["rouge1"].fmeasure
  rouge_score2 = rouge_score["rouge2"].fmeasure
  rouge_score2L = rouge_score["rougeL"].fmeasure
  return rouge_score1, rouge_score2, rouge_score2L

def remove_substring(text, substring):
    # Check if the substring is in the text
    if substring in text:
        # Replace the substring with an empty string and strip any leading/trailing spaces
        text = text.replace(substring, "").strip()
    return text

def t5_paraphrasee(text, model_id):
    # Use a regex to split sentences correctly, even if there's no space after a period
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s*', text)
    final_sentences = []

    for sentence in sentences:
        sentence = sentence.strip()
        source = tokenizer(sentence, truncation=False)
        if len(source['input_ids']) < 11 or len(source['input_ids']) > 150:
            result = [sentence]
        else:
            sentence = "Paraphrase: " + sentence
            if model_id == "m1":
              print(" -- implementing model 1 -- ")
              print("sentence : ", sentence)
              result = paraphrase_model1.paraphrase(sentence)
            else :
              print(" -- implementing model 2 -- ")
              print("sentence : ", sentence)
              result = paraphrase_model2.paraphrase(sentence)
        result = ". ".join(result)
        final_sentences.append(result.strip())

    final = ". ".join(final_sentences)
    final = re.sub(r'\.\.+', '.', final)
    final = re.sub(r'\s+\.', '.', final)
    return final

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Connectiong To Redis

In [ ]:
client = redis.Redis(
  host='redis-14401.c1.ap-southeast-1-1.ec2.redns.redis-cloud.com',
  port=14401,
  password='PELfPtyFy1UqPSROnXumtn29g5ZOmhyK')

client.ping()

True

# Program Function

In [ ]:
def create_query_table(query, queries, encoded_queries, extra_params={}):
    results_list = []
    for i, encoded_query in enumerate(encoded_queries):
        result_docs = (
            client.ft("idx:datas_vss")
            .search(
                query,
                {
                    "query_vector": np.array(
                        encoded_query, dtype=np.float32
                    ).tobytes()
                }
                | extra_params,
            )
            .docs
        )
        for doc in result_docs:
            vector_score = round(1 - float(doc.vector_score), 2)
            results_list.append(
                {
                    "query": queries[i],
                    "score": vector_score,
                    "id": doc.id,
                    "ticket_id": doc.ticket_id,
                    "subject": doc.subject,
                    "M": doc.M,
                    "R": doc.R,
                }
            )

    return results_list

In [ ]:
def displaySearchResult(results_list):
    # Optional: convert the table to Markdown using Pandas
    queries_table = pd.DataFrame(results_list)
    queries_table.sort_values(
        by=["query", "score"], ascending=[True, False], inplace=True
    )
    queries_table["query"] = queries_table.groupby("query")["query"].transform(
        lambda x: [x.iloc[0]] + [""] * (len(x) - 1)
    )
    queries_table["M"] = queries_table["M"].apply(
        lambda x: (x[:497] + "...") if len(x) > 500 else x
    )
    print(queries_table.to_markdown(index=False))
    return results_list

In [ ]:
def running_program(input, client):

    try:
        # Get the question
        queries = []
        queries.append(input)

        # embedding the question
        embedder = SentenceTransformer('/content/gdrive/MyDrive/SKRIPSI/indo-sentence-bert-base')
        encoded_queries = embedder.encode(queries)

        # initiate query by question
        query = (
            Query("(*)=>[KNN 1 @vector $query_vector AS vector_score]")
            .sort_by("vector_score")
            .return_fields('vector_score', 'id', 'ticket_id', 'subject', 'M', 'R')
            .dialect(2))

        # get the retrivied answer
        result = displaySearchResult(create_query_table(query, queries, encoded_queries))

        # paraphrased answer
        paraphrase_model = ParaphraseGenerator.load_from_checkpoint("/content/gdrive/MyDrive/SKRIPSI/T5-Model-Checkpoint/2024-05-15/t5small-epoch=09-step=5330-val_loss_mean=0.40.ckpt")
        answer_text = paraphrase_model.paraphrase(result[0]["R"])

        print(result[0]["R"])
        return result[0]["score"], result[0]["subject"], answer_text[0]
    except redis.exceptions.ConnectionError as e:
        print("Connection Error:", e)
    except redis.exceptions.TimeoutError as e:
        print("Timeout Error:", e)
    except Exception as e:
        print("Error:", e)

# TESTING BACKEND

sister tidak bisa diakses

In [ ]:
inputQuery = input("Masukkan query : ")
client = redis.Redis(
  host='redis-14401.c1.ap-southeast-1-1.ec2.redns.redis-cloud.com',
  port=14401,
  password='PELfPtyFy1UqPSROnXumtn29g5ZOmhyK')
running_program(inputQuery, client)

Masukkan query : sister tidak bisa diakses
| query                     |   score | id             |   ticket_id | subject                                      | M                                                                                                                                            | R                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

# ESTABILSH RETRIEVAL BASED WITHOUT REDIS

In [7]:
paraphrase_model1 = ParaphraseGeneratorBase.load_from_checkpoint("/content/gdrive/MyDrive/SKRIPSI/T5-Model-Checkpoint/2024-07-02/t5base-epoch=03-step=44748-val_loss_mean=0.17.ckpt")
paraphrase_model2 = ParaphraseGeneratorSmall.load_from_checkpoint("/content/gdrive/MyDrive/SKRIPSI/T5-Model-Checkpoint/2024-07-03/t5small-epoch=11-step=134244-val_loss_mean=0.21.ckpt")
embedder = SentenceTransformer('/content/gdrive/MyDrive/SKRIPSI/indo-sentence-bert-base')

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [8]:
with open("/content/gdrive/MyDrive/SKRIPSI/QA_evaluation/original_question_dataset_embedded_v4.json") as jsonfile:
  data = json.load(jsonfile)

print(len(data))
print(data[0].keys())

6622
dict_keys(['ticket_id', 'original_M', 'original_M_embedded', 'original_R'])


In [9]:
# paraphrased answer

def retrieve_answer(query, count, model_id):
  query = add_brackets(query)
  embedded_query = embedder.encode(query).astype(np.float32).tolist()
  corrent_score = 0
  best_score = 0
  top_results = []
  for y in data:
    current_score = search_similarity(embedded_query, y["original_M_embedded"])
    if len(top_results) < count:
        top_results.append((current_score, y))
        top_results = sorted(top_results, key=lambda x: x[0], reverse=True)
    elif current_score > top_results[-1][0]:
        top_results[-1] = (current_score, y)
        top_results = sorted(top_results, key=lambda x: x[0], reverse=True)

  print("len top_results: ", len(top_results))

  result_list = []
  for score, y in top_results:
      _newdict = {}
      _newdict["retrieved_id"] = y["ticket_id"]
      _newdict["query"] = remove_brackets(query)
      _newdict["retrieved_question"] = remove_brackets(y["original_M"])
      _newdict["retrieved_answer"] = remove_brackets(y["original_R"])
      _newdict["similarity_between_question"] = score
      _newdict["paraphrased_answer"] = remove_brackets(t5_paraphrasee(y["original_R"], model_id))
      _newdict["similarity_between_final_answer"] = search_similarity(
          embedder.encode(y["original_R"]).astype(np.float32).tolist(),
          embedder.encode(_newdict["paraphrased_answer"]).astype(np.float32).tolist()
      )
      final_bleu_score1, final_bleu_score2 = bleu(_newdict["paraphrased_answer"], y["original_R"])
      _newdict["final_bleu_score2"] = final_bleu_score2
      final_rouge_score1, final_rouge_score2, final_rouge_score2L = rouge(_newdict["paraphrased_answer"], y["original_R"])
      _newdict["final_rouge_score1"] = final_rouge_score1

      result_list.append(_newdict)

  return result_list

def retrieve_other_data(query, count, model_id):
  query = add_brackets(query)
  embedded_query = embedder.encode(query).astype(np.float32).tolist()
  corrent_score = 0
  best_score = 0
  top_results = []
  for y in data:
    current_score = search_similarity(embedded_query, y["original_M_embedded"])
    if len(top_results) < count:
        top_results.append((current_score, y))
        top_results = sorted(top_results, key=lambda x: x[0], reverse=True)
    elif current_score > top_results[-1][0]:
        top_results[-1] = (current_score, y)
        top_results = sorted(top_results, key=lambda x: x[0], reverse=True)

  print("len top_results: ", len(top_results))

  result_list = []
  for score, y in top_results:
      _newdict = {}
      _newdict["retrieved_id"] = y["ticket_id"]
      _newdict["query"] = remove_brackets(query)
      _newdict["retrieved_question"] = remove_brackets(y["original_M"])
      _newdict["retrieved_answer"] = remove_brackets(y["original_R"])
      _newdict["similarity_between_question"] = score
      _newdict["paraphrased_answer"] = remove_brackets(t5_paraphrasee(y["original_R"], model_id))
      _newdict["similarity_between_final_answer"] = search_similarity(
          embedder.encode(y["original_R"]).astype(np.float32).tolist(),
          embedder.encode(_newdict["paraphrased_answer"]).astype(np.float32).tolist()
      )
      final_bleu_score1, final_bleu_score2 = bleu(query, y["original_M"])
      _newdict["final_bleu_score2"] = final_bleu_score2
      final_rouge_score1, final_rouge_score2, final_rouge_score2L = rouge(query, y["original_M"])
      _newdict["final_rouge_score1"] = final_rouge_score1

      result_list.append(_newdict)

  return result_list

def define_treshold(id):
  if id == "t1":
    return 0.85
  elif id == "t2":
    return 0.75
  elif id == "t3":
    return 0.5
  elif id == "t4":
    return 0.25

In [10]:
retrieve_answer("apakah boleh konversi SKS msib namun kurang dari 20 SKS.", 1, "m1")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


len top_results:  1
 -- implementing model 1 -- 
sentence :  Paraphrase: silahkan untuk melakukan konsultasi ke operator [MBKM] fakultas.


[{'retrieved_id': '65083',
  'query': 'apakah boleh konversi SKS msib namun kurang dari 20 SKS.',
  'retrieved_question': 'NIM 201910101020. izin bertanya mengenai konversi SKS. apakah boleh konversi SKS msib namun kurang dari 20 SKS.',
  'retrieved_answer': 'silahkan untuk melakukan konsultasi ke operator MBKM fakultas.',
  'similarity_between_question': 0.8595444283954475,
  'paraphrased_answer': 'Silakan berkonsultasi dengan operator fakultas MBKM. ',
  'similarity_between_final_answer': 0.9387161238500049,
  'final_bleu_score2': 0.02777619034011792,
  'final_rouge_score1': 0.42857142857142855}]

In [11]:
answers = retrieve_answer("pemrograman krs tidak bisa diklick selesai karena tidak ada tombol selesai di sfs", 2, "m2")

len top_results:  2
 -- implementing model 2 -- 
sentence :  Paraphrase: terkait kesulitan klik selesai pada pos anda sudah kami konsultasikan dengan tim [UPTTIK] dan telah di cek.
 -- implementing model 2 -- 
sentence :  Paraphrase: Silahkan coba kembali klik selesai [KRS] anda dan mohon untuk segera menghubungi [DPA] agar dapat approve pos anda.
 -- implementing model 2 -- 
sentence :  Paraphrase: terkait kesulitan klik selesai pada pos anda sudah kami konsultasikan dengan tim [UPTTIK] dan telah di cek.
 -- implementing model 2 -- 
sentence :  Paraphrase: Silahkan coba kembali klik selesai [KRS] anda dan mohon untuk segera menghubungi [DPA] agar dapat approve pos anda.
 -- implementing model 2 -- 
sentence :  Paraphrase: Apakah anda sudah menggunakan [SFS] terbaru.


In [12]:
answers

[{'retrieved_id': '925',
  'query': 'pemrograman KRS tidak bisa diklick selesai karena tidak ada tombol selesai di SFS.',
  'retrieved_question': 'pemrograman KRS tidak bisa diklick selesai karena tidak ada tombol selesai di SFS maupun di SISTER lewat pc.',
  'retrieved_answer': 'terkait kesulitan klik selesai pada pos anda sudah kami konsultasikan dengan tim UPTTIK dan telah di cek. Silahkan coba kembali klik selesai KRS anda dan mohon untuk segera menghubungi DPA agar dapat approve pos anda. terkait kesulitan klik selesai pada pos anda sudah kami konsultasikan dengan tim UPTTIK dan telah di cek. Silahkan coba kembali klik selesai KRS anda dan mohon untuk segera menghubungi DPA agar dapat approve pos anda.',
  'similarity_between_question': 0.9079449709067317,
  'paraphrased_answer': 'Terkait kesulitan klik selesai pada pos Anda sudah kami konsultasikan dengan tim UPTTIK dan telah dia telah dia. Silahkan coba kembali klik selesai KRS Anda dan mohon untuk segera menghubungi DPA agar An

# Main Program

question list :


1.   approval penundaan pembayaran ukt tidak keluar
2.   sister tidak bisa diakses karena masa periode aktif lewat



In [ ]:
with open ("/content/gdrive/MyDrive/SKRIPSI/Question_Answering_System/data/Inferance_dataset.json") as jsonfile:
    question_data = json.load(jsonfile)

question_data[0].keys()

dict_keys(['ticket_id', 'question', 'answer'])

In [ ]:
port_no = 5000

app = Flask(__name__, template_folder='/content/gdrive/MyDrive/SKRIPSI/Question_Answering_System/templates',
            static_folder='/content/gdrive/MyDrive/SKRIPSI/Question_Answering_System/static')

ngrok.set_auth_token("2fK5CgAtJ5UBFIaGbuGqZVPIVag_7TNr6CfPH7TFphZadvwJP")
public_url =  ngrok.connect(port_no).public_url

with open ("/content/gdrive/MyDrive/SKRIPSI/Question_Answering_System/data/Inferance_dataset.json") as jsonfile:
    question_data = json.load(jsonfile)

global_response = {}

@app.route("/")
def Home():
    return render_template("index.html")

# Define a route for the prediction endpoint
@app.route('/retrieval', methods=["POST"])
def retrieval():

  print("")
  print("===== RUNNING RETRIEVAL FUNCTION =====")

  get_json = request.get_json()

  print(get_json)

  query = get_json["query"]
  query = add_brackets(query)
  treshold = get_json["choosed_treshold"]
  model = get_json["choosed_model"]

  if model == "m1": model_name = "T5-Base-0.12"
  else : model_name = "T5-Small-0.23"

  answers = retrieve_answer(query, 1, model)[0]
  retrieval_score = answers["similarity_between_question"]
  if answers["similarity_between_question"] > define_treshold(treshold) :
    status = "above_treshold"
    print_status = f"The retrieval score between query and question is above treshold: {retrieval_score}"
  else :
    status = "below_treshold"
    print_status = f"The retrieval score between query and question is below treshold: {retrieval_score}"

  print("")

  response = {
      "query": remove_brackets(query),
      "retrieved_question": answers["retrieved_question"],
      "status": status,
      "print_status": print_status,
      "retrieved_id" : answers["retrieved_id"],
      "retrieved_answer": answers["retrieved_answer"],
      "paraphrased_model": model_name,
      "paraphrased_model_id" : model,
      "paraphrased_answer": answers["paraphrased_answer"],
      "cosine_between_question": answers["similarity_between_question"],
      "final_bleu_score2": answers["final_bleu_score2"],
      "final_rouge_score1": answers["final_rouge_score1"],
  }

  global global_response
  global_response = response

  for key, value in response.items():
    print(f"{key}: {value}")

  return jsonify(response)

@app.route("/analyze", methods=["POST"])
def analyze():

  print("")
  print("===== RUNNING ANALYZE FUNCTION =====")

  get_json = request.get_json()

  ticket_id = get_json["ticket_id"]
  query = get_json["query"]
  answer = get_json["answer_text"]

  global global_response

  print(global_response)
  print(global_response.keys())

  print(ticket_id)
  print(query)

  response = {}

  for x in question_data:
    print(type(x["ticket_id"]), " and ", type(ticket_id))
    if x["ticket_id"] == str(ticket_id):
      print("retriev")
      similarity_between_question = search_similarity(
          embedder.encode(x["true_question"]).astype(np.float32).tolist(),
      embedder.encode(global_response["retrieved_question"]).astype(np.float32).tolist())
      similarity_between_answer = search_similarity(
              embedder.encode(answer).astype(np.float32).tolist(),
              embedder.encode(x["answer"]).astype(np.float32).tolist())
      response = {
          "query": global_response["query"],
          "retrieved_id" : global_response["retrieved_id"],
          "actual_id" : x["ticket_id"],
          "retrieved_question" : global_response["retrieved_question"],
          "actual_question" : remove_brackets(x["true_question"]),
          "similarity_between_question": round(similarity_between_question, 2),
          "BLEU_between_question" : round(bleu(global_response["retrieved_question"], x["true_question"])[0], 2),
          "ROUGE_between_question" : round(rouge(global_response["retrieved_question"], x["true_question"])[2], 2),
          "paraphrased_model": global_response["paraphrased_model"],
          "paraphrased_answer" : answer,
          "retrieved_answer" : global_response["retrieved_answer"],
          "actual_answer": x["answer"],
          "similarity_between_answer" : round(similarity_between_answer, 2),
          "BLEU_between_answer" : round(bleu(answer, x["answer"])[0], 2),
          "ROUGE_between_answer" : round(rouge(answer, x["answer"])[2], 2)
      }
      break
    else:
      print("not retriev")
      response = {
          "query": global_response["query"],
          "retrieved_id" : global_response["retrieved_id"],
          "actual_id" : "not retriev",
          "retrieved_question" : global_response["retrieved_question"],
          "actual_question" : remove_brackets(x["true_question"]),
          "similarity_between_question": "not retriev",
          "BLEU_between_question" : "not retriev",
          "ROUGE_between_question" : "not retriev",
          "paraphrased_model": global_response["paraphrased_model"],
          "paraphrased_answer" : answer,
          "retrieved_answer" : global_response["retrieved_answer"],
          "actual_answer": "not retriev",
          "similarity_between_answer" :"not retriev",
          "BLEU_between_answer" : "not retriev",
          "ROUGE_between_answer" : "not retriev"
      }


  for key, value in response.items():
    print(f"{key}: {value}")

  return jsonify(response)

@app.route('/get_question')
def get_question():

  print("")
  print("===== RUNNING GET QUESTION FUNCTION =====")

  response = []

  for x in question_data:
    response.append(
        {
            "ticket_id": x["ticket_id"],
            "question": remove_brackets(x["question"])
        }
    )

  random.shuffle(response)

  return jsonify(response)

@app.route('/get_other')
def get_other():

  print("")
  print("===== RUNNING GET OTHER FUNCTION =====")

  global global_response

  response = retrieve_other_data(global_response["query"], 5, global_response["paraphrased_model_id"])

  for x in response: print(x)

  return jsonify(response)

print(f"To acces the Gloable link please click {public_url}")

# Run the app if executed directly
if __name__ == "__main__":
    app.run(port=port_no)

To acces the Gloable link please click https://4d78-35-203-190-186.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [23/Jul/2024 00:47:48] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Jul/2024 00:47:49] "GET /static/script.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Jul/2024 00:47:49] "GET /static/style.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Jul/2024 00:47:49] "GET /static/table.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Jul/2024 00:47:49] "GET /static/display-direction.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Jul/2024 00:47:49] "GET /static/gif1.gif HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Jul/2024 00:47:49] "GET /static/gif2.gif HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Jul/2024 00:47:50] "GET /static/script.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Jul/2024 00:47:50] "GET /static/uc3.png HTTP/1.1" 


===== RUNNING GET QUESTION FUNCTION =====

===== RUNNING RETRIEVAL FUNCTION =====
{'query': 'Mengapa di situs SISTER tidak bisa mendaftarkan semester antara padahal penutupan semester antara tanggal 28 Juni 2019 .', 'choosed_model': 'm1', 'choosed_treshold': 't1'}
len top_results:  1
 -- implementing model 1 -- 
sentence :  Paraphrase: Setelah melakukan konfirmasi ke [BAAK] dan melihat file yang Anda lampirkan berikut .
 -- implementing model 1 -- 
sentence :  Paraphrase: Untuk [KRS] Semester Antara baru dapat dilakukan mulai tanggal 1 Juli 2019.
 -- implementing model 1 -- 
sentence :  Paraphrase: Semoga informasi ini dapat membantu.


INFO:werkzeug:127.0.0.1 - - [23/Jul/2024 00:48:22] "POST /retrieval HTTP/1.1" 200 -



query: Mengapa di situs SISTER tidak bisa mendaftarkan semester antara padahal penutupan semester antara tanggal 28 Juni 2019 .
retrieved_question: Saya mengirim pesan ini untuk menanyakan mengapa di situs SISTER tidak bisa mendaftarkan semester antara padahal penutupan semester antara tanggal 28 Juni 2019 .
status: above_treshold
print_status: The retrieval score between query and question is above treshold: 0.8940868592633148
retrieved_id: 7359
retrieved_answer: Setelah melakukan konfirmasi ke BAAK dan melihat file yang Anda lampirkan berikut . Untuk KRS Semester Antara baru dapat dilakukan mulai tanggal 1 Juli 2019. Semoga informasi ini dapat membantu.
paraphrased_model: T5-Base-0.12
paraphrased_model_id: m1
paraphrased_answer: Setelah melakukan konfirmasi ke BAAK dan melihat file yang Anda lampirkan berikut. Untuk KRS Semester Antara baru dapat dilakukan mulai tanggal 1 Juli 2019. Semoga informasi ini dapat membantu. 
cosine_between_question: 0.8940868592633148
final_bleu_score2: 

...

query: skripsi tahun depan dimulai tanggal berapa.
retrieved_id: 18689
actual_id: 7359
retrieved_question: Disini saya ingin bertanya mengenai masuk perkuliahan dimulai bulan apa dan tanggal berapa Apakah dimulai di semester selanjutnya.
actual_question: Saya mengirim pesan ini untuk menanyakan mengapa di situs SISTER tidak bisa mendaftarkan semester antara padahal penutupan semester antara tanggal 28 Juni 2019 .
similarity_between_question: 0.63
BLEU_between_question: 0.02
ROUGE_between_question: 0.15
paraphrased_model: T5-Base-0.12
paraphrased_answer: Seluruh kegiatan perkuliahan dapat dilihat dalam Kalender Akademik TA. 2020-2021 yang dapat Anda unduh melalui unduhan di website unej. Kami juga akan kirimkan Surat Edaran untuk perpanjangan kuliah daring. 
retrieved_answer: Seluruh kegiatan perkuliahan dapat anda lihat dalam Kalender Akademik TA. 2020 2021 yang dapat anda unduh melalui unduhan di web unej. Kami juga akan kirimkan Surat Edaran perpanjangan kuliah daring.
actual_answer:

INFO:werkzeug:127.0.0.1 - - [23/Jul/2024 01:43:28] "POST /retrieval HTTP/1.1" 200 -



query: selamat siang.
retrieved_question: Selamat Siang Untuk informasi dosen wali kiranya kpn nggih Terima kasih.
status: below_treshold
print_status: The retrieval score between query and question is below treshold: 0.44498112897870307
retrieved_id: 22913
retrieved_answer: Penetapan dosen wali belum dilakukan karena masa KRS bagi mahasiswa baru angkatan 2020 dimulai. Masa KRS bagi seluruh mahasiswa baru akan dilakukan secara bersamaan semua jalur masuk pada saat kegiatan PKKMB dimulai di awal September 2020.
paraphrased_model: T5-Base-0.12
paraphrased_model_id: m1
paraphrased_answer: Penetapan dosen wali belum dilakukan karena masa KRS bagi mahasiswa baru angkatan 2020 dimulai. Masa KRS bagi seluruh mahasiswa baru akan dilakukan secara bersamaan semua jalur masuk pada saat kegiatan PKKMB dimulai pada awal September 2020. 
cosine_between_question: 0.44498112897870307
final_bleu_score2: 0.7188825542773076
final_rouge_score1: 0.9722222222222222

===== RUNNING ANALYZE FUNCTION =====
{'q

INFO:werkzeug:127.0.0.1 - - [23/Jul/2024 01:58:04] "POST /analyze HTTP/1.1" 200 -


query: selamat siang.
retrieved_id: 22913
actual_id: 7359
retrieved_question: Selamat Siang Untuk informasi dosen wali kiranya kpn nggih Terima kasih.
actual_question: Saya mengirim pesan ini untuk menanyakan mengapa di situs SISTER tidak bisa mendaftarkan semester antara padahal penutupan semester antara tanggal 28 Juni 2019 .
similarity_between_question: 0.46
BLEU_between_question: 0
ROUGE_between_question: 0.06
paraphrased_model: T5-Base-0.12
paraphrased_answer: Penetapan dosen wali belum dilakukan karena masa KRS bagi mahasiswa baru angkatan 2020 dimulai. Masa KRS bagi seluruh mahasiswa baru akan dilakukan secara bersamaan semua jalur masuk pada saat kegiatan PKKMB dimulai pada awal September 2020. 
retrieved_answer: Penetapan dosen wali belum dilakukan karena masa KRS bagi mahasiswa baru angkatan 2020 dimulai. Masa KRS bagi seluruh mahasiswa baru akan dilakukan secara bersamaan semua jalur masuk pada saat kegiatan PKKMB dimulai di awal September 2020.
actual_answer: Setelah melaku

================================================

In [ ]:
global_response

{}

In [ ]:
# OLD DATA

@app.route("/analyze", methods=["POST"])
def analyze():
  print("analyze section")
  get_json = request.get_json()

  ticket_id = get_json["ticket_id"]
  query = get_json["query"]
  answer = get_json["answer_text"]

  global global_response

  print(ticket_id)
  print(query)

  response = {}

  for x in question_data:
    print(type(x["ticket_id"]), " and ", type(ticket_id))
    if x["ticket_id"] == str(ticket_id):
      print("retriev")
      similarity_between_question = search_similarity(
          embedder.encode(query).astype(np.float32).tolist(),
      embedder.encode(x["question"]).astype(np.float32).tolist())
      similarity_between_answer = search_similarity(
              embedder.encode(answer).astype(np.float32).tolist(),
              embedder.encode(x["answer"]).astype(np.float32).tolist())
      response = {
          "retrieved_id" : global_response["retrieved_id"],
          "actual_id" : x["ticket_id"],
          "retrieved_question" : query,
          "actual_question" : x["question"],
          "similarity_between_question": round(similarity_between_question, 2),
          "BLEU_between_question" : round(bleu(query, x["question"])[0], 2),
          "ROUGE_between_question" : round(rouge(query, x["question"])[2], 2),
          "answer" : answer,
          "actual_answer": x["answer"],
          "similarity_between_answer" : round(similarity_between_answer, 2),
          "BLEU_between_answer" : round(bleu(answer, x["answer"])[0], 2),
          "ROUGE_between_answer" : round(rouge(answer, x["answer"])[2], 2)
      }
      break
    else: print("not retriev")


  for key, value in response.items():
    print(f"{key}: {value}")

  return jsonify(response)